# Класификација на АДХД преку ЕЕГ сигнали (претпроцесирање)

In [182]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mne
import os
import seaborn as sns
from glob import glob
import warnings
from autopreprocess_pipeline import *
from autopreprocessing import dataset as ds
import shutil 
from tqdm import tqdm

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv1D, MaxPool1D, Dense, Flatten, Dropout

### Вчитување, филтрирање на пациенти од интерес и отстранување артефакти (?)

In [2]:
main_path = '/home/lukar/projects/eeg/data'

Податочното множество кое се користи во проектот е дел од опширната ЕЕГ база на податоци наречена Two Decades-Brainclinics Research Archive for Insights in Neurophysiologyсе (TDBRAIN), кое се состои од датотеки (кои се именувани со уникатна шифра на пациентот: participants_ID). Секоја датотека содржи најмалку 2 (во ретки случаеви и 3) снимања на пациент кој може да е здрав или да боледува од психичко растројство. Во овој случај не' интересираат само пациентите обележани со HEALTHY и ADHD. Датотеката 'derivatives', која претходно ги содржела пациентите со разни растројства е филтрирана и понатаму користена во кодот во следните чекори. 

In [3]:
df = pd.read_csv(os.path.join(main_path, 'TDBRAIN_ID_and_status.csv')) # convert the .xlsx file into a .csv file beforehand
df_subset = df[['participants_ID', 'formal_status']] # only participants' ID and their status are needed from all columns
df_filtered = df_subset[df_subset['formal_status'].isin(['HEALTHY', 'ADHD'])] # out of the 5+ statuses (classes), only healthy and adhd ones are needed

filtered_file_path = '/home/lukar/projects/eeg/data/dataTDBRAIN_ID_and_status.csv'
df_filtered.to_csv(filtered_file_path, index=False) # save the .csv file

In [4]:
df_filtered
df_filtered.head()

,participants_ID,formal_status
0,sub-87974617,HEALTHY
1,sub-87974621,HEALTHY
2,sub-87974665,HEALTHY
3,sub-87974709,HEALTHY
4,sub-87974841,HEALTHY


In [ ]:
text = 'Fp1,Fp2,F7,F3,Fz,F4,F8,FC3,FCz,FC4,T7,C3,Cz,C4,T8,CP3,CPz,CP4,P7,P3,Pz,P4,P8,O1,Oz,O2,VPVA,VNVB,HPHL,HNHR,Erbs,OrbOcc,Mass' # electrode names copied from one .csv eeg recordings file
channel_names = text.split(',')  # split electrode names with commas
channel_names = [f"{name}" for name in channel_names] # create a list of strings 
print(channel_names)

In [ ]:
folders_path = '/home/lukar/projects/eeg/data/derivatives' # Снимките од пациентите пред да се претпроцесираат/отстранат артефакти

In [ ]:
sourcepath = folders_path
preprocpath = '/home/lukar/projects/eeg/data/processed_subjects' # Филтрирани и исчистени снимки кои ќе се користат во понатамошната работа.  

In [ ]:
varargsin = {
    'sourcepath' : folders_path,
    'preprocpath' : preprocpath
}

За претпроцесирање на податоците е искористен алгоритмот препорачан и тестиран од тимот на Brainclinics. Кодот е достапен заедно со податочното множество (на страната на Brainclinics), но оригиналната верзија е модифицирана согласно тековните верзии на наредбите во неа. Претпроцесирањето се извршува со објекти од класата dataset, кој ја содржи ЕЕГ снимката согласно форматирањето на Brainclinics, заедно со функции за корекција на артефакти и опција за зачувување на обработените снимки како .csv, .npy или .mat датотеки. 

За секое снимање, функцијата <code>autoprocess_standard</code> ќе ги детектира: 
1. ЕОГ (очни) артефакти - потекнуваат од движење на очите нагоре-надолу, лево-десно и од трепкање 
1. ЕМГ артефакти - потекнуваат од мускулна активност 
1. Сегментите со екстремна зашиленост (анг. <i>kurtosis</i>) 
1. Невообичаените скокови и разлики во максималниот позитивен и негативен напон (анг. <i>voltage swings</i>)
1. Артефакти кои потекнуваат од трепкање

Доколку повеќе од третина од податоците по канал е означена како артефакт, каналот е означен за да му се изврши интерполација врз основа на сигналите снимени од двете соседни електроди. Дополнително, каналите со шум низ целиот спектар и со ЕМГ примеси долж целиот сигнал се означуваат како неквалитетни. За каналите кај кои се јавуваат проблеми како празни вредности и преклопување со соседна електрода поради вишок спроводлив гел се врши интерполација со податоци од соседните канали.


In [ ]:
autopreprocess_standard(varargsin=varargsin) # Функција за отстранување на артефакти предложена од научниците кои го составиле податочното множество.  

### Поделба на пациенти и сегментирање на сигналите 

In [ ]:
def chop_string_from_end(string, char, flip): # Помошна функција за форматирање на имињата на датотеките кои ќе се добијат по сегментирањето на податоците. 
    
    index = string.rfind(char)
    if not flip:
        if index != -1:
            chopped_string = string[index:]
        else:
            chopped_string = string

        return chopped_string
    else:
        if index != -1:
            chopped_string = string[:index]
        else:
            chopped_string = string

        return chopped_string



За да се добијат доволно поединечни точки во податочното множество, наместо да се користат целите ЕЕГ снимки, тие се сегментирани со преклопувачки лизгачки прозорец.
Секое снимање на пациент трае 120 секунди, и снимките се семплирани со фреквенција од 500 Hz. Ова значи дека секоја (целосна) снимка ќе има 60000 податочни точки, и во конечната верзија на проектот таа ќе биде сегментирана со прозорец од големина 5000 точки и преклоп од 500 точки.

Бројот на сегменти кој го добиваме од секоја снимка е даден со следната формула:
$$
    N = \left\lfloor \frac{T - W}{W - O} \right\rfloor + 1 \\ 
$$
Каде: <br><br>
$
    W - \text{Големина на прозорец} \\
    T - \text{Вкупен број временски точки во оригинална снимка} \\
    O - \text{Должина на преклопување} \\
$
<br><br>

Ова соодветствува на: $t_{window} = \frac{5000}{500Hz} = 10s$, и преклоп од $1s$

Ако ги замениме вредностите кои ги одбравме за нашите податоци, тогаш се добива $N = 13$ како број на нови сегменти добиени од секоја оригинална снимка. 
<br>
НАПОМЕНА: во текот на изработката на проектот се пробувани различни големини за лизгачкиот прозорец со цел да се балансира меѓу добивката на големината на податочното множество, и квалитетот/информациите кои ги содржи секоја податочна точка. Доколку искористиме премал чекор и прозорец, ќе имаме голем број податоци но тие ќе носат помалку информации во секоја точка, додека во обратниот случај податоците ќе се поквалитетни/информативни, но ќе имаме премалку за тренирање добар модел, и носење на значаен заклучок. Заклучено е дека големините на сегмените опишани горе се најдобри, но бидејќи тетратката е користена многу пати, како и поединечните ќелии во неа, можно е output-ите на некои ќелии да не се совпаѓаат со други. 

In [ ]:
def segment_csv(path_to_file, path_to_dir, window_length=5000, overlap=500):

    df = pd.read_csv(path_to_file)
    
    df = df.drop(columns=['artifacts', 'VEOG', 'HEOG', 'Erbs', 'OrbOcc', 'Mass'], axis=1) # Се отстрануваат колоните кои се останати од иницијалното чистење на артефактите во сигналите
    i=0 
    for i in range(13):
        sub_df = df.iloc[i*overlap : i*overlap + window_length]
        i+=1
        subject_name = chop_string_from_end(path_to_file,"/", flip=0)
        clean_name = chop_string_from_end(subject_name,"eeg_csv",flip=1)

        seg_path = path_to_dir + "/" + clean_name + "_seg_" + str(i) + ".csv"

        
        
        sub_df.to_csv(seg_path)       
    

In [ ]:
path_to_dir = "/home/lukar/projects/eeg/data/processed_subjects"

In [ ]:

def find_csv_files(directory): # помошна функција која ни овозможува да добиеме листа од имиња на датотеки кои ќе бидат сегментирани
    """
    Recursively searches for .csv files in the given directory and its subdirectories.
    Returns a list of paths to the found .csv files.
    """
    csv_files = []

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".csv"):
                csv_files.append(os.path.join(root, file))
                
    return csv_files            


In [10]:
df_dict = df_filtered.set_index('participants_ID')['formal_status'].to_dict()

Прво, целосните снимки (датотеки) ги делиме на 2 - една датотека со пациенти кои се дијагностицирани со АДХД, и дадотека на пациенти без дијагноза. 

In [11]:
source_dir = '/home/lukar/projects/eeg/data/processed_subjects'
target_dir_adhd = '/home/lukar/projects/eeg/data/adhd'
target_dir_healthy = '/home/lukar/projects/eeg/data/healthy'

In [ ]:

for subject in os.listdir(source_dir): # преместување на измешани пациенти во различни датотеки 
   

    if df_dict[subject] == "HEALTHY":
        shutil.move(os.path.join(source_dir, subject), target_dir_healthy)
    elif df_dict[subject] == "ADHD":
        shutil.move(os.path.join(source_dir, subject), target_dir_adhd)   
                          


In [13]:
adhd_dir = os.listdir(target_dir_adhd)
healthy_dir = os.listdir(target_dir_healthy)

Ги чуваме патеките на секоја датотека на пациент, понатаму ќе ги поделиме во множества за тренирање, тестирање, и валидација 

In [14]:
adhd_dir = [os.path.join(target_dir_adhd,subject) for subject in adhd_dir]
healthy_dir = [os.path.join(target_dir_healthy,subject) for subject in healthy_dir]

In [15]:
training_path = '/home/lukar/projects/eeg/data/training'
validation_path = '/home/lukar/projects/eeg/data/validation'
testing_path = '/home/lukar/projects/eeg/data/testing'

Во податочното множество, пациентите се поделени на следниот начин:
 - Tренирање: 31 HEALTHY, 31 ADHD 
 - Валидација: 4 HEALTHY, 4 ADHD
 - Тестирање: 4 HEALTHY, 4 ADHD

In [ ]:
for i in range (len(healthy_dir)): 
    if i<31: # alternate between healthy and adhd subject and add 31 samples of each into training data 
        shutil.move(healthy_dir[i], training_path)
        shutil.move(adhd_dir[i], training_path)
    elif i >= 31 and i < 35:  # alternate between healthy and adhd subject and add 4 samples of each into validation data 
        shutil.move(healthy_dir[i], validation_path)
        shutil.move(adhd_dir[i], validation_path)
    elif i>=35 and i < 39: # alternate between healthy and adhd subject and add 4 samples of each into testing data
        shutil.move(healthy_dir[i], testing_path)
        shutil.move(adhd_dir[i], testing_path)

In [ ]:
training_csv_files = find_csv_files(training_path) # ги чуваме патеките за снимањата на секој пациент 
validation_csv_files = find_csv_files(validation_path)
testing_csv_files = find_csv_files(testing_path)

In [ ]:
training_segmented_path =   "/home/lukar/projects/eeg/data/training_segmented" # ги чуваме сегментираните датотеки во посебни директориуми 
validation_segmented_path =  "/home/lukar/projects/eeg/data/validation_segmented"
testing_segmented_path = "/home/lukar/projects/eeg/data/testing_segmented"

In [20]:
for training_file in tqdm(training_csv_files):
    segment_csv(path_to_file = training_file, path_to_dir = training_segmented_path)

100%|█████████████████████████████████████████████████████████████████████████████████| 124/124 [17:46<00:00,  8.60s/it]


In [21]:
for validation_file in tqdm(validation_csv_files):
    segment_csv(path_to_file = validation_file, path_to_dir = validation_segmented_path)
   

100%|███████████████████████████████████████████████████████████████████████████████████| 36/36 [05:06<00:00,  8.52s/it]


In [22]:
for testing_file in tqdm(testing_csv_files):
    segment_csv(path_to_file = testing_file, path_to_dir = testing_segmented_path) 

100%|███████████████████████████████████████████████████████████████████████████████████| 34/34 [04:48<00:00,  8.48s/it]


### Извлекување на карактеристики и поделба во множества за тренирање, тестирање и валидација

Во овој дел се извлекуваат статистички и спектрални обележја од секој сегмент, и се зачувуваат како .npy датотеки кои ќе се користат во понатамошната анализа и класификација. 

Изборот на обележјата е инспириран од слични истражувања во оваа област:

- Коефициент на асиметрија, зашиленост [E. Sathiya, T. D. Rao, and T. S. Kumar, “Gabor filter-based statistical features for ADHD detection,” Front. Hum. Neurosci., vol. 18, p. 1369862, Apr. 2024, doi: <a>10.3389/fnhum.2024.1369862.</a>]
- Јортови параметри [B. Hjorth, “EEG analysis based on time domain properties,” Electroencephalography and Clinical Neurophysiology, vol. 29, no. 3, pp. 306–310, Sep. 1970, doi: <a>10.1016/0013-4694(70)90143-4.</a>]
- Моќности на алфа, бета, гама, делта и тета опсезите [M. Moghaddari, M. Z. Lighvan, and S. Danishvar, “Diagnose ADHD disorder in children using convolutional neural network based on continuous mental task EEG,” Computer Methods and Programs in Biomedicine, vol. 197, p. 105738, Dec. 2020, doi: <a>10.1016/j.cmpb.2020.105738.</a>]
- Нелинеарни обележја (фрактална димензија според алгоритмите на: Хигучи, Јорт и Петросијан) [M. R. Mohammadi, A. Khaleghi, A. M. Nasrabadi, S. Rafieivand, M. Begol, and H. Zarafshan, “EEG classification of ADHD and normal children using non-linear features and neural network,” Biomed. Eng. Lett., vol. 6, no. 2, pp. 66–73, May 2016, doi: <a>10.1007/s13534-016-0218-2.</a>]
- Мерки за спектрална ентропија [T. Inouye et al., “Quantification of EEG irregularity by use of the entropy of the power spectrum,” Electroencephalography and Clinical Neurophysiology, vol. 79, no. 3, pp. 204–210, Sep. 1991, doi: <a>10.1016/0013-4694(91)90138-T.</a>]

In [ ]:
training_filepaths = [os.path.join(training_segmented_path,file) for file in os.listdir(training_segmented_path)]
validation_filepaths = [os.path.join(validation_segmented_path,file) for file in os.listdir(validation_segmented_path)]
testing_filepaths = [os.path.join(testing_segmented_path,file) for file in os.listdir(testing_segmented_path)]

X_train = []
X_val = []
X_test = [] 

y_train = []
y_val = []
y_test = [] 

Конкретно, статистичките обележја кои се извлекуваат од секој сигнал се следните: 
1. Средна вредност 
1. Ефективна вредност (RMS)
1. Стандардна девијација 
1. Коефициент на  асиметрија (анг. <i>skewness</i>) 
1. Зашиленост (анг. <i>kurtosis</i>)

Јортовите параметри: 
1. Активност - мерка за варијансата на амплитудата на сигналот 
1. Мобилност - мерката за стандардната девијација на нагибот (изводот) на сигналот во однос на стандардната девијација на амплитудата
1. Комплексност -  сооднос меѓу мобилноста на првиот извод на сигналот и мобилноста на самиот сигнал

Спектрални обележја:
1. Алфа-моќност
1. Бета-моќност 
1. Делта-моќност 
1. Тета-моќност
1. Гама-моќност 
1. Однос на тета и бета моќности 
1. Шенонова ентропија
1. Спектрална ентропија за тета и бета опсези 
1. Вејвлет ентропија

Нелинеарни обележја:
1. Фрактална димензија според Хичуги 
1. Фрактална димензија според Кац 
1. Фрактална димензија според Петросијан 


In [141]:
import numpy as np
import pywt
from scipy.spatial.distance import pdist
from scipy.stats import entropy
from sklearn.neighbors import NearestNeighbors

def wavelet_entropy(segment, wavelet='db4', level=4):
    coeffs = pywt.wavedec(segment, wavelet, level=level)
    entropy_list = []
    for coeff in coeffs:
        coeff = np.abs(coeff)
        norm_coeff = coeff / np.sum(coeff)
        entropy_list.append(entropy(norm_coeff))
    return np.mean(entropy_list)


import numpy as np
import pandas as pd
from scipy.stats import kurtosis, skew, entropy
from scipy.signal import welch
from scipy.integrate import simpson
from sklearn.preprocessing import StandardScaler
from numpy import log2, mean, sqrt
from math import log10

def compute_bandpower(segment, fs, band):
    freqs, psd = welch(segment, fs)
    band_freqs = (freqs >= band[0]) & (freqs <= band[1])
    band_power = simpson(y=psd[band_freqs], x=freqs[band_freqs])
    return band_power, psd, freqs

def hjorth_params(segment):
    # Hjorth Activity
    activity = np.var(segment)
    
    # Hjorth Mobility
    derivative = np.diff(segment)
    mobility = np.std(derivative) / np.std(segment)
    
    # Hjorth Complexity
    second_derivative = np.diff(derivative)
    complexity = (np.std(second_derivative) / np.std(derivative)) / mobility
    
    return activity, mobility, complexity

def spectral_entropy(psd):
    psd_norm = psd / np.sum(psd)
    return entropy(psd_norm)

def shannon_entropy(segment):
    prob_dist, _ = np.histogram(segment, bins=256, density=True)
    prob_dist = prob_dist[prob_dist > 0]
    return -np.sum(prob_dist * np.log2(prob_dist))

def higuchi_fd(segment, k_max):
    L = []
    x = np.asarray(segment)
    N = len(x)

    for k in range(1, k_max):
        Lk = 0
        for m in range(k):
            Lmk = 0
            for i in range(1, int(np.floor((N - m) / k))):
                Lmk += np.abs(x[m + i * k] - x[m + (i - 1) * k])
            Lmk = Lmk * (N - 1) / (int(np.floor((N - m) / k)) * k)
            Lk += Lmk
        L.append(np.log(Lk / k))

    return np.polyfit(np.log(range(1, k_max)), L, 1)[0]

    
def katz_fd(segment):
    L = np.sum(np.sqrt(np.ediff1d(segment) ** 2 + 1))
    d = np.max(np.abs(segment - segment[0]))
    N = len(segment)
    return log10(L) / (log10(d) + log10(N))

def petrosian_fd(segment):
    n = len(segment)
    diff = np.diff(segment)
    N_delta = np.sum(diff[1:] * diff[:-1] < 0)
    return log10(n) / (log10(n) + log10(n / (n + 0.4 * N_delta)))


In [ ]:

def extract_features(df, fs,eo):
    feature_list = []

    for column in df.columns:
        segment = df[column].values
        # Compute time-domain features
        mean_val = np.mean(segment)
        std_val = np.std(segment)
        rms_val = np.sqrt(np.mean(segment**2))
        kurtosis_val = kurtosis(segment)
        skewness_val = skew(segment)

        # Compute Hjorth parameters
        activity, mobility, complexity = hjorth_params(segment)

        # Compute Shannon's entropy
        shannon_entropy_val = shannon_entropy(segment)

        # Compute band powers and PSD entropy
        delta_power, _, _ = compute_bandpower(segment, fs, [0.5, 4])
        theta_power, psd_t, freqs = compute_bandpower(segment, fs, [4, 8])
        alpha_power, _, _ = compute_bandpower(segment, fs, [8, 13])
        beta_power, psd_b, _ = compute_bandpower(segment, fs, [13, 30])
        gamma_power, _, _ = compute_bandpower(segment, fs, [30, 100])

        
        
        # Compute spectral entropy for theta and beta bands
        spectral_entropy_theta = spectral_entropy(psd_t)
        spectral_entropy_beta = spectral_entropy(psd_b)

        # Compute wavelet entropy
        wavelet_entropy_val = wavelet_entropy(segment)
        
        # Compute theta to beta power ratio
        theta_beta_ratio = theta_power/beta_power

        # Combine all features
        features = [
            mean_val, std_val, rms_val, kurtosis_val, skewness_val,
            activity, mobility, complexity, shannon_entropy_val, spectral_entropy_theta, spectral_entropy_beta,
            delta_power, theta_power, alpha_power, beta_power, gamma_power, wavelet_entropy_val, theta_beta_ratio
        ]
        feature_list.append(features)

    feature_array = np.array(feature_list)

    
    features_flattened = feature_array.flatten()

    
   
    return features_flattened



За патеките во директориумите за тренирање, валидација, и тестирање, ги пресметуваме вредностите за обележјата наведени погоре, податоците ги зачувуваме во .npy датотеки

In [ ]:
for csv in tqdm(training_filepaths): 

    
    index = csv.rfind("/")
    subject_name = csv[index+1:]
    subject_index = subject_name.find("_")
    subject_name = subject_name[:subject_index]
   

    if df_dict[subject_name] == "HEALTHY":
        y_train.append(0)
    elif df_dict[subject_name] == "ADHD":
         y_train.append(1)

    
    segment = pd.read_csv(csv)
    segment = segment.drop(columns = ['Unnamed: 0.1', 'Unnamed: 0'])
    
    feature_file = extract_features(df = segment,fs = 500)
    

    
    X_train.append(feature_file)

print (f"Training data loaded! Training set: {len(X_train)}, labels: {len(y_train)}")

100%|█████████████████████████████████████████████████████████████████████████████| 13640/13640 [15:17<00:00, 14.86it/s]

Training data loaded! Training set: 13640, labels: 13640


In [ ]:
for csv in tqdm(validation_filepaths): 
    
    index = csv.rfind("/")
    subject_name = csv[index+1:]
    subject_index = subject_name.find("_")
    subject_name = subject_name[:subject_index]
   

    if df_dict[subject_name] == "HEALTHY":
        y_val.append(0)
    elif df_dict[subject_name] == "ADHD":
         y_val.append(1)

    
    segment = pd.read_csv(csv)
    segment = segment.drop(columns = ['Unnamed: 0.1', 'Unnamed: 0'])
    
    feature_file = extract_features(df = segment,fs = 500)
        
    
        
    X_val.append(feature_file)

print (f"Validation data loaded! Validation set: {len(X_val)}, labels: {len(y_val)}")

100%|███████████████████████████████████████████████████████████████████████████████| 3960/3960 [04:23<00:00, 15.01it/s]

Validation data loaded! Validation set: 3960, labels: 3960


In [ ]:
for csv in tqdm(testing_filepaths): 
    
    index = csv.rfind("/")
    subject_name = csv[index+1:]
    subject_index = subject_name.find("_")
    subject_name = subject_name[:subject_index]
   

    if df_dict[subject_name] == "HEALTHY":
        y_test.append(0)
    elif df_dict[subject_name] == "ADHD":
         y_test.append(1)

    
    segment = pd.read_csv(csv)
    segment = segment.drop(columns = ['Unnamed: 0.1', 'Unnamed: 0'])
    feature_file = extract_features(df = segment, fs = 500)
  
    X_test.append(feature_file)

print (f"Testing data loaded! Testing set: {len(X_test)}, labels: {len(y_test)}")

100%|███████████████████████████████████████████████████████████████████████████████| 3740/3740 [04:10<00:00, 14.90it/s]

Testing data loaded! Testing set: 3740, labels: 3740


In [153]:
np.save("X_train.npy", X_train)
np.save("X_test.npy", X_test)
np.save("X_val.npy", X_val)
np.save("y_train.npy", y_train)
np.save("y_test.npy", y_test)
np.save("y_val.npy", y_val)